<a href="https://colab.research.google.com/github/phillipSloan/ComputationalLogic/blob/main/Report%20Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Report for COMSM0022 Computational Logic for Artificial Intelligence (2021)

## created by [Phillip Sloan](https://gibhub.com/phillipSloan) and [Jonathan Erskine](https://github.com/jmerskine1)

### Introduction
In this coursework we attempt to extend the reasoning capabilities of Prolexa to include negation.

This report demonstrates our approach to the assignment, walking through how we implemented negation and other command line code. It seeks to explain our thought process, which was wrong at certain points, causing unwanted proofs from the meta-interpreter. There are several additions to the default prolexa code detailed in the report, with notebook cells demonstrating implementation and operation.


### Instantiation of Prolexa for Notebook Demonstration

In [ ]:
!apt-get install swi-prolog -qqq > /dev/null

In [ ]:
!yes | pip install --upgrade git+https://github.com/phillipSloan/ComputationalLogic/ -qqq > /dev/null

from pyswip import Prolog
import prolexa.meta_grammar as meta

# Added this due to an error with meta
import nltk
nltk.download('omw-1.4')
  

pl = Prolog()
meta.reset_grammar()
meta.initialise_prolexa(pl)

# clearing the output to keep it tidy
from IPython.display import clear_output 
clear_output()



### Negation

In it's current state, Prolexa cannot handle negation semantically, or in terms of reasoning e.g. given a statement "Tweety does not fly" or "Tweety is not a bird", Prolexa will fail to interpret the natural language of the query due to the unknown effects of "not" within sentence structure, and it cannot associate "not" with any meaning regarding a clause or set of clauses.




**Grammar**

To implement negation grammatically we have to modify prolexa_grammar.pl to define a "not" operator, (taken from Simply Logical 8.1) and to include negated verb phrases:
```julia
:-op(900,fy,not).

verb_phrase(s,not(M)) --> [is],[not],property(s,M).
verb_phrase(s,not(M)) --> [not],property(s,M).
```
Introducing "not(M)" into the verb_phrase requires us to extend our definition of sentence1 to include negative cases:
```julia
sentence1([(H:-not(B))]) --> determiner(N,M1,M2,[(H:-B)]),noun(N,M1),verb_phrase(N,not(M2)).
```
This implements a special case where, if the verb phrase is negative, we pass the negative rule but borrow the standard determiner from the positive case. The modification is more straight forward for proper nouns:
```julia
sentence1([(not(L):-true)]) --> proper_noun(N,X),verb_phrase(N,not(X=>L)).
```

Here we have only dealt with the singular case, so negated phrases like "All teachers are not happy" aren't currently handled by Prolog - this can be replaced with "Every teacher is not happy" so we will not attempt to extend the grammar for the purposes of this demonstration. 

Finally, we need to extend the question interpreter to understand "not" within a query:
```julia
question1(not(Q)) --> [who],verb_phrase(s,not(_X=>Q)).
question1(not(Q)) --> [is],proper_noun(N,X),verb_phrase(N,not(X=>Q)).
```

---
<font color='red'>Demonstration</font>: *Understanding negated phrases*

In [ ]:
print(meta.standardised_query(pl, "donald is not happy")[0]['Output'])
print(meta.standardised_query(pl, "every teacher is not immortal")[0]['Output'])

print(meta.standardised_query(pl, "spill the beans")[0]['Output'])

I will remember that donald is not happy
I will remember that every teacher is not immortal
donald is not happy. donald is not happy. every teacher is not immortal. every teacher is not immortal


Note the **duplication of negated rules**. This behavior is not replicated in standard prolexa and we were unable to locate the issue. However, the following code proves correct handling of rules, and performance does not seem to be affected.

---



**Reasoning**

We can now handle phrases like "Donald is not happy" and "Every teacher is not happy", but they have no bearing with respect to reasoning. This can be observed if we input some conflicting information:

```
user: "tell me everything".
prolexa: I know nothing

user: "donald is happy".
prolexa: I will remember that donald is happy

user: "donald is not happy".
prolexa: I will remember that donald is not happy

user: "tell me everything".
prolexa: donald is happy. donald is not happy
```
Prolexa cannot recognise the confliction between donald being happy and unhappy ("not happy") at the same time. To enable this we need to apply a function which takes a rule and searches the current rulebase to remove any which are in direct conflict. The following is added to prolexa_engine.pl :
```julia
remove_conflicting_rules([Head:-Body]):-
	(conflicting_not_rules(Head:-Body)
	; retractall(prolexa:stored_rule(_,[(not(Head):-Body)])),
	     retractall(prolexa:stored_rule(_,[(Head:-not(Body))])) ).

conflicting_not_rules(not(Head):-Body):-
	retractall(prolexa:stored_rule(_,[(Head:-Body)])).

conflicting_not_rules(Head:-not(Body)):-
	retractall(prolexa:stored_rule(_,[(Head:-Body)])).
```
This function is called from prolexa.pl when a new rule is added.
```
% A. Utterance is a sentence
	( phrase(sentence(Rule),UtteranceList),
	  write_debug(rule(Rule)),
	  ( known_rule(Rule,SessionId) -> % A1. It follows from known rules
			atomic_list_concat(['I already knew that',Utterance],' ',Answer)
	  ; otherwise -> % A2. It doesn't follow, so add to stored rules
```
```julia
		        remove_conflicting_rules(Rule),
```
```
			assertz(prolexa:stored_rule(SessionId,Rule)),
			atomic_list_concat(['I will remember that',Utterance],' ',Answer)
	  )
```


---
<font color='red'>Demonstration</font>: *Removing Conflicting Rules*

In [ ]:
print('Establish Rulebase:')
print(meta.standardised_query(pl, "forget everything")[0]['Output'])
print(meta.standardised_query(pl, "donald is not happy")[0]['Output'])
print(meta.standardised_query(pl, "every teacher is immortal")[0]['Output'])
print(meta.standardised_query(pl, "spill the beans")[0]['Output'])

print('\n Overwrite current rules with conflicting rules:')
print(meta.standardised_query(pl, "donald is happy")[0]['Output'])
print(meta.standardised_query(pl, "every teacher is not immortal")[0]['Output'])
print(meta.standardised_query(pl, "spill the beans")[0]['Output'])

Establish Rulebase:
b'I am a blank slate'
I will remember that donald is not happy
I will remember that every teacher is immortal
donald is not happy. donald is not happy. every teacher is immortal

 Overwrite current rules with conflicting rules:
I will remember that donald is happy
I will remember that every teacher is not immortal
donald is happy. every teacher is not immortal. every teacher is not immortal


---

Prolexa can now properly store and remove rules by considering new, conflicting information, but we can still not infer answers to negated questions from positive literals, or vice versa. For example:
```
user: "donald is not happy".
prolexa: I will remember that donald is not happy

user: "is donald happy"
prolexa: Sorry, I don't think this is the case
```
and 
```
user: "donald is happy".
prolexa: I will remember that donald is not happy

user: "is donald not happy"
prolexa: Sorry, I don't think this is the case
```
The question answering engine will attempt to prove the query, and if it cannot will provide a response indicating that the answer is not found in the knowledge base ("Sorry, I don't..."). 

However, in this case, clearly "Is donald happy?" should have an answer as we know that Donald is not happy. To remedy this, we add an extra step in the question answering process to check if the negative of a query can be proven. 

This requires modification of prove_question/2, prove_question/3 and explain_question to duplicate the first check but with the negative version of the query.

```
%%% Main question-answering engine adapted from nl_shell.pl %%%

prove_question(Query,SessionId,Answer):-
    findall(R,prolexa:stored_rule(SessionId,R),Rulebase),
    ( prove_rb(Query,Rulebase) ->
        transform(Query,Clauses),
        phrase(sentence(Clauses),AnswerAtomList),
        atomics_to_string(AnswerAtomList," ",Answer)
```
```julia
    ; prove_rb(not Query,Rulebase) ->
        transform(not Query,Clauses),
        phrase(sentence(Clauses),AnswerAtomList),
        atomics_to_string(AnswerAtomList," ",Answer)
```
```
    ; Answer = 'Sorry, I don\'t think this is the case'
    ).
    
% two-argument version that can be used in maplist/3 (see all_answers/2)
prove_question(Query,Answer):-
	findall(R,prolexa:stored_rule(_SessionId,R),Rulebase),
	( prove_rb(Query,Rulebase) ->
		transform(Query,Clauses),
		phrase(sentence(Clauses),AnswerAtomList),
		atomics_to_string(AnswerAtomList," ",Answer)
```
```julia
	; prove_rb(not Query,Rulebase) ->
			transform(not Query,Clauses),
			phrase(sentence(Clauses),AnswerAtomList),
			atomics_to_string(AnswerAtomList," ",Answer)
```
```
	; Answer = ""
	).


%%% Extended version of prove_question/3 that constructs a proof tree %%%
explain_question(Query,SessionId,Answer):-
	findall(R,prolexa:stored_rule(SessionId,R),Rulebase),
	( prove_rb(Query,Rulebase,[],Proof) ->
		maplist(pstep2message,Proof,Msg),
		phrase(sentence1([(Query:-true)]),L),
		atomic_list_concat([therefore|L]," ",Last),
		append(Msg,[Last],Messages),
		atomic_list_concat(Messages,"; ",Answer)
```
```julia
	; prove_rb(not(Query),Rulebase,[],Proof) ->
		maplist(pstep2message,Proof,Msg),
		phrase(sentence1([(not(Query):-true)]),L),
		atomic_list_concat([therefore|L]," ",Last),
		append(Msg,[Last],Messages),
		atomic_list_concat(Messages," ; ",Answer)
```
```
	; Answer = 'Sorry, I don\'t think this is the case'
	).

```

Prolog can now handle our first example with a correct answer:
```
user: "tell me everything".
prolexa: donald is not happy

user:  "is donald happy".
prolexa: donald is not happy
```
However, the second example still fails:
```
user: "tell me everything".
prolexa: donald is happy

user:  "is donald not happy".
prolexa: Sorry, I don't think this is the case
```
Investigating the issue, it becomes apparent that this type of question passes double negative queries to prolexa's question answering engine, so we need to extend the meta-interpreter to understand that 

> not(not(A)) --> A

We add the following to prove_rb:
```
%for double negatives
prove_rb(not(not(A)),Rulebase,P0,P):-
  find_clause((A:-B),Rule,Rulebase),
	prove_rb(B,Rulebase,[p(A,Rule)|P0],P).
```
Prolog can now reason with double negatives, but we need to adapt the transform predicate to simplify double negatives to positives for answer generation:
```
% transform instantiated, possibly conjunctive, query to list of clauses
transform((A,B),[(A:-true)|Rest]):-!,
    transform(B,Rest).
transform(not(not(A)),B):-!,
	transform(A,B).
transform(A,[(A:-true)]).
```

---
<font color='red'>Demonstration</font>: **Inferring answers to negated queries from positive literals (and vice versa)*

In [ ]:
print(meta.standardised_query(pl, "forget everything")[0]['Output'])
print(meta.standardised_query(pl, "donald is happy")[0]['Output'])
print(meta.standardised_query(pl, "tell me everything")[0]['Output'])

print(meta.standardised_query(pl, "is donald not happy")[0]['Output'])

b'I am a blank slate'
I will remember that donald is happy
donald is happy
b'donald is happy'


In [ ]:
print(meta.standardised_query(pl, "forget everything")[0]['Output'])
print(meta.standardised_query(pl, "donald is not happy")[0]['Output'])
print(meta.standardised_query(pl, "tell me everything")[0]['Output'])

print(meta.standardised_query(pl, "is donald happy")[0]['Output'])

b'I am a blank slate'
I will remember that donald is not happy
donald is not happy. donald is not happy
b'donald is not happy'



___
The assignment asks us to prove:
> Every teacher is happy. Donald is not happy. Therefore, Donald is not a teacher.

This test fails, yielding the following dialogue:
```
user: "tell me everything".
prolexa: donald is not happy. every teacher is happy

user: "explain why donald is not a teacher".
prolexa: Sorry, I don't think this is the case
```
Clearly, our reasoning methods are still falling short. If donald is happy, then we can not say for certain whether he is or is not a teacher. However, knowing that donald is not happy confirms that he cannot be a teacher, as all teachers are happy.

When looking at the final predicate, it can be seen that prove_rb/4 tries to unify B with a body of a stored rule whos head matches the given A. It will cycle through all stored rules and try and find a matching rule, but in the second case this will not happen, as it will be trying to match teacher(donald) to happy(X), so it fails. We realised that a new predicate needed to be created, that mirrored the existing one and unified the head A, given a body B. The following predicate was added to prove_rb/4.:
```
prove_rb(true,_Rulebase,P,P):-!.
prove_rb((A,B),Rulebase,P0,P):-!,
    find_clause((A:-C),Rule,Rulebase),
    conj_append(C,B,D),
    prove_rb(D,Rulebase,[p((A,B),Rule)|P0],P).
prove_rb(A,Rulebase,P0,P):-
    find_clause((A:-B),Rule,Rulebase),
    prove_rb(B,Rulebase,[p(A,Rule)|P0],P).
```
```julia
prove_rb(not B,Rulebase,P0,P):-
  find_clause((A:-B),Rule,Rulebase),
	prove_rb(not A,Rulebase,[p(not B,Rule)|P0],P).
```

The extension of prove_rb/4 takes the negation of what is put into it, which can be seen with `not B`. `find_clause/4` finds all clauses which unify a head `A` to given body `B` (i.e. find_clause/4 would find `happy(X):-teacher(X)` from `teacher(donald)`, unifying to create `happy(donald):-teacher(donald)`). Finally, a recursive call seeks to prove `not A` (which, with our example would then be seeking to unify `not(happy(donald))` to a relevent rule in the rulebase. This works because because A:-B is equivalent to not(B):-not(A).


We can now infer that donald is not a teacher, demonstrating an extension of the capability of prolexa to reason using negation.

---
<font color='red'>Demonstration</font>: **Inferring answers to queries by reasoning through negation*

In [ ]:
print('Establish knowledge base:')
print(meta.standardised_query(pl, "forget everything")[0]['Output'])
print(meta.standardised_query(pl, "every teacher is happy")[0]['Output'])
print(meta.standardised_query(pl, "donald is not happy")[0]['Output'])

print('\n Perform test:')
print(meta.standardised_query(pl, "explain why donald is not a teacher")[0]['Output'])

Establish knowledge base:
b'I am a blank slate'
I will remember that every teacher is happy
I will remember that donald is not happy

 Perform test:
donald is not happy; every teacher is happy; therefore donald is not a teacher


---

<font color='red'>Demonstration</font>: *Additional Question-Answer Capabilities*

In [ ]:
print('Establish knowledge base:')
print(meta.standardised_query(pl, "forget everything")[0]['Output'])
print(meta.standardised_query(pl, "every teacher is happy")[0]['Output'])
print(meta.standardised_query(pl, "donald is not happy")[0]['Output'])


print("Who is... / Is Donald ... examples")
print(meta.standardised_query(pl, "who is not a teacher")[0]['Output'])
print(meta.standardised_query(pl, "is donald not a teacher")[0]['Output'])

Establish knowledge base:
b'I am a blank slate'
I will remember that every teacher is happy
I will remember that donald is not happy
Who is... / Is Donald ... examples
b'donald is not a teacher'
b'donald is not a teacher'


### For further testing

In [ ]:
input = ''  #@param {type:"string"}
print(input)
first_answer = meta.standardised_query(pl, input)[0]['Output']
print(first_answer)

Tell me about donald"
donald is not happy. donald is not a teacher. donald is not happy. donald is not a teacher
